In [4]:
%pip install huggingface_hub

   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   ----------- ---------------------------- 41.0/144.7 kB 2.0 MB/s eta 0:00:01
   ---------------- ---------------------- 61.4/144.7 kB 825.8 kB/s eta 0:00:01
   ---------------- ---------------------- 61.4/144.7 kB 825.8 kB/s eta 0:00:01
   ------------------- ------------------- 71.7/144.7 kB 438.9 kB/s eta 0:00:01
   ------------------- ------------------- 71.7/144.7 kB 438.9 kB/s eta 0:00:01
   ------------------- ------------------- 71.7/144.7 kB 438.9 kB/s eta 0:00:01
   ------------------- ------------------- 71.7/144.7 kB 438.9 kB/s eta 0:00:01
   ------------

In [2]:
import os
import cv2
import timm
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
from torch.autograd import Function
from torchvision import transforms

plt.rcParams["figure.figsize"] = (20,20)

In [5]:
class FeatureExtractor():
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x

class ModelOutputs():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model, feature_module, target_layers):
        self.model = model
        self.feature_module = feature_module
        self.feature_extractor = FeatureExtractor(self.feature_module, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations = []
        for name, module in self.model._modules.items():
            if module == self.feature_module:
                target_activations, x = self.feature_extractor(x)
            elif "avgpool" in name.lower():
                x = module(x)
                x = x.view(x.size(0),-1)
            else:
                x = module(x)

        return target_activations, x

def preprocess_image(img):

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    preprocessing = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((224, 224))
                    normalize,
    ])
    return preprocessing(img.copy()).unsqueeze(0)

def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

class GradCam:
    def __init__(self, model, feature_module, target_layer_names, use_cuda):
        self.model = model
        self.feature_module = feature_module
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = ModelOutputs(self.model, self.feature_module, target_layer_names)

    def forward(self, input_img):
        return self.model(input_img)

    def __call__(self, input_img, target_category=None):
        if self.cuda:
            input_img = input_img.cuda()

        features, output = self.extractor(input_img)

        if target_category == None:
            target_category = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][target_category] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = one_hot.cuda()

        one_hot = torch.sum(one_hot * output)

        self.feature_module.zero_grad()
        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

        target = features[-1]
        target = target.cpu().data.numpy()[0, :]

        weights = np.mean(grads_val, axis=(2, 3))[0, :]
        cam = np.zeros(target.shape[1:], dtype=np.float32)

        for i, w in enumerate(weights):
            cam += w * target[i, :, :]

        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, input_img.shape[2:])
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam


class GuidedBackpropReLU(Function):
    @staticmethod
    def forward(self, input_img):
        positive_mask = (input_img > 0).type_as(input_img)
        output = torch.addcmul(torch.zeros(input_img.size()).type_as(input_img), input_img, positive_mask)
        self.save_for_backward(input_img, output)
        return output

    @staticmethod
    def backward(self, grad_output):
        input_img, output = self.saved_tensors
        grad_input = None

        positive_mask_1 = (input_img > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = torch.addcmul(torch.zeros(input_img.size()).type_as(input_img),
                                   torch.addcmul(torch.zeros(input_img.size()).type_as(input_img), grad_output,
                                                 positive_mask_1), positive_mask_2)
        return grad_input


class GuidedBackpropReLUModel:
    def __init__(self, model, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        def recursive_relu_apply(module_top):
            for idx, module in module_top._modules.items():
                recursive_relu_apply(module)
                if module.__class__.__name__ == 'ReLU':
                    module_top._modules[idx] = GuidedBackpropReLU.apply

        # replace ReLU with GuidedBackpropReLU
        recursive_relu_apply(self.model)

    def forward(self, input_img):
        return self.model(input_img)

    def __call__(self, input_img, target_category=None):
        if self.cuda:
            input_img = input_img.cuda()

        input_img = input_img.requires_grad_(True)

        output = self.forward(input_img)

        if target_category == None:
            target_category = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][target_category] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = one_hot.cuda()

        one_hot = torch.sum(one_hot * output)
        one_hot.backward(retain_graph=True)

        output = input_img.grad.cpu().data.numpy()
        output = output[0, :, :, :]

        return output

def deprocess_image(img):
    """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
    img = img - np.mean(img)
    img = img / (np.std(img) + 1e-5)
    img = img * 0.1
    img = img + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img*255)

SyntaxError: ignored

In [ ]:
use_cuda = torch.cuda.is_available()
MODEL_PATH = "/content/drive/MyDrive/GradCam/VGG-16.pt"
IMAGE_PATH = "/content/drive/MyDrive/GradCam/GradCam Raw/esophagitis-b-d.jpg"

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Define your VGG16 model
model = models.vgg16(pretrained=False)

# Modify the classifier to match your task (assuming 5 classes)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 5)

# Load the model checkpoint using torch.jit.load (if it's in TorchScript format)
MODEL_PATH = '/content/drive/MyDrive/GradCam/VGG-16.pt'
use_cuda = True  # Change this to True if you want to use GPU

# Load the model using torch.jit.load
loaded_model = torch.jit.load(MODEL_PATH)

if use_cuda:
    loaded_model.to('cuda')
else:
    loaded_model.to('cpu')

In [ ]:
# Load an image using OpenCV
img = cv2.imread(IMAGE_PATH, 1)
img = np.float32(img) / 255
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_img = preprocess_image(img)  # Make sure you have a preprocess_image function

# Grad-CAM
grad_cam = GradCam(model, model.features, ['29'], use_cuda=use_cuda)

# If None, it returns the map for the highest scoring category.
# Otherwise, you can set the target_category to a specific class.
target_category = None
grayscale_cam = grad_cam(input_img, target_category)

grayscale_cam = cv2.resize(grayscale_cam, (img.shape[1], img.shape[0]))
cam = show_cam_on_image(img, grayscale_cam)

# Guided Backpropagation
gb_model = GuidedBackpropReLUModel(model, use_cuda=use_cuda)
gb = gb_model(input_img, target_category=target_category)
gb = gb.transpose((1, 2, 0))

# Save the visualization results
cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
cam_gb = deprocess_image(cam_mask * gb)
gb = deprocess_image(gb)

cv2.imwrite("cam.jpg", cam)
cv2.imwrite("gb.jpg", gb)
cv2.imwrite("cam_gb.jpg", cam_gb)

In [ ]:
import cv2
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
cam = cv2.imread("./cam.jpg")
cam = cv2.cvtColor(cam, cv2.COLOR_BGR2RGB)
ax.imshow(cam)
plt.show()


In [11]:
import imgaug.augmenters as iaa
import os
import random
from PIL import Image
import numpy as np

# Set the path to the folder containing your original images
input_folder = "C:/Users/ungab/OneDrive/Pictures/Tumor Bed Selection Dataset/Non/"

# Set the path to the folder where augmented images will be saved
output_folder = "C:/Users/ungab/OneDrive/Desktop/Shakhawat Sir's Work/aug_train/"

# Define the augmentation pipeline
augmentation_pipeline = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),  # horizontally flip 50% of the images
    iaa.Affine(rotate=(-45, 45)),
        # rotate images by -45 to 45 degrees
])

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through each class folder
for class_folder in os.listdir(input_folder):
    class_folder_path = os.path.join(input_folder, class_folder)
    # Check if the item in the folder is a subdirectory
    if os.path.isdir(class_folder_path):
        output_class_folder_path = os.path.join(output_folder, class_folder)
        # Create the output subfolder for the class if it doesn't exist
        if not os.path.exists(output_class_folder_path):
            os.makedirs(output_class_folder_path)
        # Get the list of images in the class folder
        image_files = [file for file in os.listdir(class_folder_path) if file.endswith(".jpg") or file.endswith(".png")]
        # Determine the number of images to be created for this class folder
        num_original_images = len(image_files)
        num_augmented_images_needed = max(1000 - num_original_images, 0)  # Ensure the minimum is 0
        num_augmented_images_needed = min(num_augmented_images_needed, 1000)  # Ensure the maximum is 1000
        # Loop to create additional images for this class folder
        for i in range(num_augmented_images_needed):
            # Randomly select an image from the class folder
            random_image = random.choice(image_files)
            image_path = os.path.join(class_folder_path, random_image)
            # Open the image and convert it to a NumPy array
            image = np.array(Image.open(image_path))
            # Apply augmentation to create an additional image
            augmented_image = augmentation_pipeline(images=[image])[0]
            # Convert the augmented NumPy array back to PIL Image
            augmented_image = Image.fromarray(augmented_image)
            # Save the augmented image to the output folder
            output_image_path = os.path.join(output_class_folder_path, f"augmented_{i}_{random_image}")
            augmented_image.save(output_image_path)


In [16]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import imgaug.augmenters as iaa

def apply_augmentation(input_path, output_path, augmentation_factor=2000):
    # Ensure output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # List all files in the input directory
    input_files = os.listdir(input_path)

    # Define augmentation sequence
    augmenter = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Affine(rotate=(-45, 45)),
        # Add more augmentations as needed
    ])

    for file_name in tqdm(input_files, desc='Applying Augmentation'):
        # Load the original image
        img_path = os.path.join(input_path, file_name)
        img = cv2.imread(img_path)

        # Apply augmentation and save augmented images
        for i in range(augmentation_factor):
            augmented_img = augment_image(img, augmenter)
            augmented_file_name = f"{os.path.splitext(file_name)[0]}_aug_{i + 1}{os.path.splitext(file_name)[1]}"
            augmented_file_path = os.path.join(output_path, augmented_file_name)
            cv2.imwrite(augmented_file_path, augmented_img)

def augment_image(img, augmenter):
    # Apply imgaug augmentation
    augmented_img = augmenter.augment_image(img)
    return augmented_img

# Example usage
input_directory = "C:/Users/ungab/OneDrive/Pictures/Tumor Bed Selection Dataset/"
output_directory = "C:/Users/ungab/OneDrive/Desktop/Shakhawat Sir's Work/aug_train/"

# Specify the desired number of augmented images per class
augmentation_factor_per_class = 2000

# Loop through subdirectories (assuming each subdirectory represents a class)
for class_folder in os.listdir(input_directory):
    class_input_path = os.path.join(input_directory, class_folder)
    class_output_path = os.path.join(output_directory, class_folder)

    apply_augmentation(class_input_path, class_output_path, augmentation_factor_per_class)


Applying Augmentation:   0%|          | 1/214 [04:39<16:31:51, 279.40s/it]


KeyboardInterrupt: 

In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import imgaug.augmenters as iaa

def apply_augmentation(input_path, output_path, target_count=3000):
    # Ensure output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # List all files in the input directory
    input_files = os.listdir(input_path)

    # Define augmentation sequence
    augmenter = iaa.Sequential([
    #iaa.Fliplr(),
    #iaa.Flipud(),
    iaa.Affine(rotate=(15, -15)),
    #iaa.Affine(scale=(1.5, 2)),
    #iaa.contrast.LinearContrast((1.5, 2.5)),
    #iaa.AdditiveGaussianNoise(scale=(0, 0.1*255)),  # Add Gaussian noise
    #iaa.Multiply((0.8, 1.2)),  # Multiply pixel values for brightness variation
    #iaa.GammaContrast((1.0, 2.0)),  # Adjust gamma for contrast
    #iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),  # Sharpen images
    #iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 3.0))),  # Apply Gaussian blur randomly
    # Add more augmentations as needed
    ])

    # Calculate how many augmentations are needed for each original image
    augmentations_per_image = max(2, target_count // len(input_files))

    for file_name in tqdm(input_files, desc='Applying Augmentation'):
        # Load the original image
        img_path = os.path.join(input_path, file_name)
        img = cv2.imread(img_path)

        # Apply augmentation and save augmented images
        for i in range(augmentations_per_image):
            augmented_img = augment_image(img, augmenter)
            augmented_file_name = f"{os.path.splitext(file_name)[0]}_aug_{i + 1}{os.path.splitext(file_name)[1]}"
            augmented_file_path = os.path.join(output_path, augmented_file_name)
            cv2.imwrite(augmented_file_path, augmented_img)

def augment_image(img, augmenter):
    # Apply imgaug augmentation
    augmented_img = augmenter.augment_image(img)
    return augmented_img

# Example usage
input_directory = "C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion/"
output_directory = "C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion3/"
# Specify the desired number of augmented images per class
target_augmentation_count_per_class = 3000

# Loop through subdirectories (assuming each subdirectory represents a class)
for class_folder in os.listdir(input_directory):
    class_input_path = os.path.join(input_directory, class_folder)
    class_output_path = os.path.join(output_directory, class_folder)

    apply_augmentation(class_input_path, class_output_path, target_augmentation_count_per_class)


Applying Augmentation:  70%|███████   | 35/50 [00:17<00:07,  2.05it/s]


KeyboardInterrupt: 

In [15]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import imgaug.augmenters as iaa

def apply_augmentation(input_path, output_path, total_images_per_class=3000, augmentation_factor=2):
    # Ensure output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # List all files in the input directory
    input_files = os.listdir(input_path)

    # Calculate the number of augmented images needed
    images_needed = total_images_per_class

    # Adjust augmentation factor if needed
    augmentation_factor = min(augmentation_factor, images_needed)

    # Define augmentation sequence
    augmenter = iaa.Sequential([
        #iaa.Sharpen(alpha=(0.75, 0.75)),
        iaa.Fliplr(),
        iaa.Affine(rotate=(-15, 15))  # Sharpen images
        #  Add more augmentations as needed
        #, lightness=(0.5, 0.5)
    ])

    for file_name in tqdm(input_files, desc=f'Processing {output_path}'):
        # Load the original image
        img_path = os.path.join(input_path, file_name)
        img = cv2.imread(img_path)

        # Apply augmentation and save augmented images
        for i in range(augmentation_factor):
            augmented_img = augment_image(img, augmenter)
            augmented_file_name = f"{os.path.splitext(file_name)[0]}_aug_{i + 1}{os.path.splitext(file_name)[1]}"
            augmented_file_path = os.path.join(output_path, augmented_file_name)
            cv2.imwrite(augmented_file_path, augmented_img)

def augment_image(img, augmenter):
    # Apply imgaug augmentation
    augmented_img = augmenter.augment_image(img)
    return augmented_img

# Example usage
input_directory = "C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion4/"
output_directory = "C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion5/"
# Specify the desired number of total images per class (augmented only)
total_images_per_class = 3000

# Loop through subdirectories (assuming each subdirectory represents a class)
for class_folder in os.listdir(input_directory):
    class_input_path = os.path.join(input_directory, class_folder)
    class_output_path = os.path.join(output_directory, class_folder)

    apply_augmentation(class_input_path, class_output_path, total_images_per_class)


Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion5/actinic keratosis: 100%|██████████| 50/50 [00:01<00:00, 41.17it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion5/basal cell carcinoma: 100%|██████████| 50/50 [00:01<00:00, 45.88it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion5/benign keratosis: 100%|██████████| 50/50 [00:03<00:00, 16.66it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion5/dermatofibroma: 100%|██████████| 50/50 [00:01<00:00, 46.81it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion5/melanocytic nevus: 100%|██████████| 50/50 [00:10<00:00,  4.89it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The Intern

In [1]:
import os
import cv2
from tqdm import tqdm
import imgaug.augmenters as iaa

def apply_augmentation(input_path, output_path, total_images_per_class=2000, augmentation_factor=2):
    # Ensure output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # List all files in the input directory
    input_files = os.listdir(input_path)

    # Calculate the number of augmented images needed
    images_needed = total_images_per_class

    # Adjust augmentation factor if needed
    augmentation_factor = min(augmentation_factor, images_needed)

    # Define augmentation sequence
    augmenter = iaa.Sequential([
        #iaa.Fliplr(0.5),
        # iaa.Flipud(0.5),
        #iaa.Affine(rotate=(-10, 10)),
         iaa.Sharpen(alpha=(0, 0.25))  # Adding shearing
        # Add more augmentations as needed
    ])

    for file_name in tqdm(input_files, desc=f'Processing {output_path}'):
        # Load the original image
        img_path = os.path.join(input_path, file_name)
        img = cv2.imread(img_path)

        # Apply augmentation and save augmented images
        for i in range(augmentation_factor):
            augmented_img = augment_image(img, augmenter)
            augmented_file_name = f"{os.path.splitext(file_name)[0]}_aug_{i + 1}{os.path.splitext(file_name)[1]}"
            augmented_file_path = os.path.join(output_path, augmented_file_name)
            cv2.imwrite(augmented_file_path, augmented_img)

def augment_image(img, augmenter):
    # Apply imgaug augmentation
    augmented_img = augmenter.augment_image(img)
    return augmented_img

# Example usage
input_directory = "C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion/"
output_directory = "C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion1/"
# Specify the desired number of total images per class (augmented only)
total_images_per_class = 2000

# Loop through subdirectories (assuming each subdirectory represents a class)
for class_folder in os.listdir(input_directory):
    class_input_path = os.path.join(input_directory, class_folder)
    class_output_path = os.path.join(output_directory, class_folder)

    apply_augmentation(class_input_path, class_output_path, total_images_per_class)


Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion1/actinic keratosis: 100%|██████████| 50/50 [00:01<00:00, 33.01it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion1/basal cell carcinoma: 100%|██████████| 50/50 [00:01<00:00, 33.71it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion1/benign keratosis: 100%|██████████| 50/50 [00:03<00:00, 15.54it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion1/dermatofibroma: 100%|██████████| 50/50 [00:01<00:00, 29.70it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The International Skin Imaging Collaboration/Skin_Lesion1/melanocytic nevus: 100%|██████████| 50/50 [00:10<00:00,  4.66it/s]
Processing C:/Users/ungab/Downloads/archive/Skin cancer ISIC The Intern

In [5]:
import os
import random
import shutil

def random_select_images(source_dir, destination_dir, num_images=3000):
    # Get a list of all files in the source directory
    all_images = [file for file in os.listdir(source_dir) if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]

    # Ensure the number of requested images is not greater than the total number of images
    num_images = min(num_images, len(all_images))

    # Randomly select images
    selected_images = random.sample(all_images, num_images)

    # Copy selected images to the destination directory
    for image in selected_images:
        source_path = os.path.join(source_dir, image)
        destination_path = os.path.join(destination_dir, image)
        shutil.copy2(source_path, destination_path)

    print(f"{num_images} images randomly selected and copied to {destination_dir}.")

# Replace 'source_directory_path' and 'destination_directory_path' withyour actual directory paths
source_directory_path = 'C:/Users/ungab/Downloads/Compressed/results/BCC/'
destination_directory_path = 'C:/Users/ungab/Downloads/Compressed/BCC/'

random_select_images(source_directory_path, destination_directory_path, num_images=2000)


2000 images randomly selected and copied to C:/Users/ungab/Downloads/Compressed/BCC/.


In [1]:
import cv2
import numpy as np

def remove_hair(image_path, output_path):
    # Read the image
    img = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply a median blur to reduce noise
    blur = cv2.medianBlur(gray, 5)

    # Use adaptive thresholding to create a binary mask
    _, thresh = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY)

    # Use morphological operations to enhance the mask
    kernel = np.ones((5, 5), np.uint8)
    morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Invert the mask
    inv_mask = cv2.bitwise_not(morphed)

    # Apply the mask to the original image
    result = cv2.bitwise_and(img, img, mask=inv_mask)

    # Save the result
    cv2.imwrite(output_path, result)

# Example usage
input_image_path = 'C:/Users/ungab/Downloads/Compressed/ISIC 2019/BCC/ISIC_0072693.jpg'
output_image_path = 'C:/Users/ungab/Downloads/Compressed/result.jpg'

remove_hair(input_image_path, output_image_path)


In [2]:
from PIL import Image
import os

def convert_jpg_to_png(input_dir, output_dir):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # List all files in the input directory
    input_files = os.listdir(input_dir)

    for file_name in input_files:
        if file_name.endswith(".jpg") or file_name.endswith(".jpeg"):
            # Form the complete path for the input and output files
            input_path = os.path.join(input_dir, file_name)
            output_path = os.path.join(output_dir, os.path.splitext(file_name)[0] + ".png")

            # Open the JPG image and save it as PNG
            try:
                img = Image.open(input_path)
                img.save(output_path, "PNG")
                print(f"Converted: {file_name}")
            except Exception as e:
                print(f"Error converting {file_name}: {e}")

if __name__ == "__main__":
    # Provide your input and output directories
    input_directory = "C:/Users/ungab/Downloads/Compressed/kvasir-seg/Kvasir-SEG/masks/"
    output_directory = "C:/Users/ungab/Downloads/Compressed/kvasir-seg/Kvasir-SEG/masks1/"

    convert_jpg_to_png(input_directory, output_directory)


Converted: cju0qkwl35piu0993l0dewei2.jpg
Converted: cju0qoxqj9q6s0835b43399p4.jpg
Converted: cju0qx73cjw570799j4n5cjze.jpg
Converted: cju0roawvklrq0799vmjorwfv.jpg
Converted: cju0rx1idathl0835detmsp84.jpg
Converted: cju0s2a9ekvms080138tjjpxr.jpg
Converted: cju0s690hkp960855tjuaqvv0.jpg
Converted: cju0sr5ghl0nd08789uzf1raf.jpg
Converted: cju0sxqiclckk08551ycbwhno.jpg
Converted: cju0t4oil7vzk099370nun5h9.jpg
Converted: cju0tl3uz8blh0993wxvn7ly3.jpg
Converted: cju0u2g7pmnux0801vkk47ivj.jpg
Converted: cju0u82z3cuma0835wlxrnrjv.jpg
Converted: cju0ue769mxii08019zqgdbxn.jpg
Converted: cju0vtox5ain6099360pu62rp.jpg
Converted: cju13cgqmnhwn0988yrainhcp.jpg
Converted: cju13fwthn9mq0835gacxgy01.jpg
Converted: cju13hp5rnbjx0835bf0jowgx.jpg
Converted: cju14g8o4xui30878gkgbrvqj.jpg
Converted: cju14hjh2ob2o0835ouz3r5aa.jpg
Converted: cju14pxbaoksp0835qzorx6g6.jpg
Converted: cju15czxqp3lv0835jvhgzurz.jpg
Converted: cju15jr8jz8sb0855ukmkswkz.jpg
Converted: cju15l5ubz9yh0855b3ivdpse.jpg
Converted: cju15

In [9]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_dataset(dataset_path, output_path, test_size=0.1, random_state=42):
    # Create output directories
    train_path = os.path.join(output_path, 'train')
    test_path = os.path.join(output_path, 'test')

    os.makedirs(train_path, exist_ok=True)
    os.makedirs(test_path, exist_ok=True)

    # Get list of image and mask files
    image_files = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]
    mask_files = [f for f in os.listdir(dataset_path) if f.endswith('.png')]

    # Sort to ensure correspondence between images and masks
    image_files.sort()
    mask_files.sort()

    # Check if the dataset is large enough for splitting
    if len(image_files) < 2 or len(mask_files) < 2:
        print("Dataset is too small to split. Please make sure you have enough images and masks.")
        return

    # Split the dataset
    image_train, image_test, mask_train, mask_test = train_test_split(image_files, mask_files, test_size=test_size, random_state=random_state)

    # Move training set to the train directory
    for image_file, mask_file in zip(image_train, mask_train):
        shutil.copy(os.path.join(dataset_path, image_file), os.path.join(train_path, image_file))
        shutil.copy(os.path.join(dataset_path, mask_file), os.path.join(train_path, mask_file))

    # Move testing set to the test directory
    for image_file, mask_file in zip(image_test, mask_test):
        shutil.copy(os.path.join(dataset_path, image_file), os.path.join(test_path, image_file))
        shutil.copy(os.path.join(dataset_path, mask_file), os.path.join(test_path, mask_file))

if __name__ == "__main__":
    # Set your dataset path and output path
    dataset_path = "C:/Users/ungab/Downloads/Compressed/kvasir-seg/"
    output_path = "C:/Users/ungab/Downloads/Compressed/kvasir-seg/"

    # Call the split_dataset function
    split_dataset(dataset_path, output_path)


Dataset is too small to split. Please make sure you have enough images and masks.


In [1]:
import cv2
import json
import numpy as np
 
 
def draw_bounding_boxes(image_path, annotations, output_path):
    image = cv2.imread(image_path)
    masks = np.zeros_like(image)  # Initialize an empty array to store masks
 
    for _, region_data in annotations.items():
        for _, region_info in region_data['regions'].items():
            shape_attributes = region_info['shape_attributes']
            label = region_info['region_attributes']['label']
            points_x = shape_attributes['all_points_x']
            points_y = shape_attributes['all_points_y']
 
            # Convert the list of points into a list of tuples (x, y)
            points = [(int(x), int(y)) for x, y in zip(points_x, points_y)]
 
            # Create a mask for the region
            region_mask = np.zeros_like(image)
            cv2.fillPoly(region_mask, [np.array(points)], (255, 255, 255))
 
            # Combine the mask with the overall masks
            masks = cv2.add(masks, region_mask)
 
            # Draw the polygon on the original image
            cv2.polylines(image, [np.array(points)], isClosed=True, color=(0, 255, 0), thickness=2)
 
            # Draw the label
            x, y = points[0]  # Fix the unpacking of the first point (x, y)
            cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
 
    # Save the mask and annotated image
    cv2.imwrite(output_path + '_mask1.png', masks)
    cv2.imwrite(output_path + '_annotated1.jpg', image)
 
if __name__ == "__main__":
    image_path = "C:/Users/ungab/Downloads/421145203_369282889047984_4835365853749445881_n.jpg" # Update with the correct path to your image
    vgg_json_path = "C:/Users/ungab/Downloads/labels_my-project-name_2024-01-29-10-34-47.json"  # Update with the correct path to your JSON file
    output_prefix = "C:/Users/ungab/Downloads/"  # Update with the desired output path prefix
 
    with open(vgg_json_path) as f:
        annotations = json.load(f)
 
    draw_bounding_boxes(image_path, annotations, output_prefix)